<a href="https://colab.research.google.com/github/Shreepoorna1903/pesu-connect_ui/blob/main/reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Sentiment Labels

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents.  As we discussed in class, many classification models represent a document as a sparse vector of of features and their weights. Weights could be boolean, counts, tf-idf values, or other functions of the document. In a linear model, we then combine the document vectors with the candidate output classes and learn weights for each of the feature-class pairs. This last step is implemented for you.

For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

There are many helpful resources online for getting up to speed with vector representations of documents. One example is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [2]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np

In [3]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [6]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.
import re
from collections import Counter
from scipy.sparse import coo_matrix

Letter_present = re.compile(r".*[A-Za-z].*") # Regex value to keep tokens with 1 ASSCI charecter.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.

def letters_to_be_tokenized(text):
    tokens = re.findall(r"\b\w+\b", text.lower()) # Taking word chunks and turning them into simple words
    return [t for t in tokens if Letter_present.fullmatch(t)]

def createBasicFeatures(corpus):
    raw_texts = [ex["text"]  for ex in corpus] #iterating over the corpus and segregating on basis of text and label
    classes   = [ex["label"] for ex in corpus]

    tokenizable_words = [letters_to_be_tokenized(t) for t in raw_texts]
    vocab = sorted({tok for toks in tokenizable_words for tok in toks})
    token_to_col = {tok: i for i, tok in enumerate(vocab)}


    rows, cols, data = [], [], []
    for i, toks in enumerate(tokenizable_words):  # Building matrix in triplet form as coo_matrix from scipy uses coordinate format (row, col, value) triples.
        counts = Counter(toks)
        for tok, cnt in counts.items():
            rows.append(i)
            cols.append(token_to_col[tok])
            data.append(cnt)


    texts = coo_matrix((data, (rows, cols)),
                       shape=(len(raw_texts), len(vocab)),
                       dtype=float).tocsr()

    return texts, classes, vocab

In [ ]:
# TODO: Implement createFancyFeatures andn describe in comments what the
# features are and why they might ben helpful.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.

import re
from collections import Counter
from scipy.sparse import coo_matrix, hstack
import math
from sklearn.feature_extraction.text import TfidfTransformer

def Add_bigramsFeature(tokens):
    return [f"{a}_{b}" for a, b in zip(tokens, tokens[1:])] # building adjacent word bigrams

def Add_ngramsFeature(token, min=3, max=5): # building adjacent word n grams ranging between 3 to 5
    output = []
    s = token
    L = len(s)
    for n in range(min, max + 1):
        if L >= n:
            output.extend(s[i:i+n] for i in range(L - n + 1))
    return output

def add_tfidf_feature(X_counts):
    tfidf = TfidfTransformer(norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=True)
    return tfidf.fit_transform(X_counts)

def build_block(docs_features):
    vocab = sorted({f for feats in docs_features for f in feats})
    f2c = {f: i for i, f in enumerate(vocab)}
    rows, cols, data = [], [], []
    for r, feats in enumerate(docs_features):
        counts = Counter(feats)
        for f, c in counts.items():
            rows.append(r); cols.append(f2c[f]); data.append(c)
    X = coo_matrix((data, (rows, cols)), shape=(len(docs_features), len(vocab)), dtype=float).tocsr()
    return X, vocab

def createFancyFeatures(corpus, use_tfidf=True,ngramsPresent=True):
    raw_texts = [ex["text"]  for ex in corpus] #iterating over the corpus and segregating on basis of text and label
    classes   = [ex["label"] for ex in corpus]

    tokenizable_words = [letters_to_be_tokenized(t) for t in raw_texts]

    unigram_documents = tokenizable_words
    X_unigram, V_unigram = build_block(unigram_documents)
    V_unigram = [f"uni:{w}" for w in V_unigram]

    bigram_documents = [Add_bigramsFeature(toks) for toks in tokenizable_words]
    X_bigram, V_bigram = build_block(bigram_documents)
    V_bigram = [f"bi:{w}" for w in V_bigram]

    X_word = hstack([X_unigram, X_bigram], format="csr")
    V_word = V_unigram + V_bigram

    if ngramsPresent:
        char_docs = [[c for tok in toks for c in Add_ngramsFeature(tok, 3, 5)] for toks in tokenizable_words]
        X_char, V_char = build_block(char_docs)
        V_char = [f"char:{w}" for w in V_char]
        X_all = hstack([X_word, X_char], format="csr")
        V_all = V_word + V_char
    else:
        X_all = X_word
        V_all = V_word

    texts = add_tfidf_feature(X_all) if use_tfidf else X_all
    vocab = V_all

    return texts, classes, vocab

In [ ]:
# given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))

  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [4]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [ ]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

Run the following to train and evaluate two models using extended features:

In [ ]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

**TODO**: Briefly comment on your results. You do _not_ need to beat the basic features model to get full credit.